### Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Load synthetic pairs

In [4]:
pairs_df = pd.read_csv("synth_pairs.csv")

print("Shape:", pairs_df.shape)
print("Label distribution:\n", pairs_df["label"].value_counts())

Shape: (400, 7)
Label distribution:
 label
1    200
0    200
Name: count, dtype: int64
